# Nama Kelompok :
1. Achmad Fadlan Bhahreisy (06.2018.1.06992)
2. Dimas Pratikto (06.2018.1.06958)
3. Muhammad Ihsan Nugroho (06.2018.1.07058)
4. M. Ilham Afriandhi (06.2018.1.06943)

## 1. Request to website

In [1]:
import requests
import string
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup


# Make a request to the website
r = requests.get('https://www.kompas.com/')
# Create an object to parse the HTML format
soup = BeautifulSoup(r.content, 'html.parser')
# Retrieve all popular news links (Fig. 1)
link = []
for i in soup.find('div', {'class':'most__wrap'}).find_all('a'):
    i['href'] = i['href'] + '?page=all'
    link.append(i['href'])
# For each link, we retrieve paragraphs from it, combine each paragraph as one string, and save it to documents (Fig. 2)
documents = []
for i in link:
    # Make a request to the link
    r = requests.get(i)
  
    # Initialize BeautifulSoup object to parse the content 
    soup = BeautifulSoup(r.content, 'html.parser')
  
    # Retrieve all paragraphs and combine it as one
    sen = []
    for i in soup.find('div', {'class':'read__content'}).find_all('p'):
        sen.append(i.text)
  
    # Add the combined paragraphs to documents
    documents.append(' '.join(sen))


## 2. Regex

In [2]:
import re
documents_clean = []
for d in documents:
    # Remove Unicode
    document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    # Lowercase the document
    document_test = document_test.lower()
    # Remove punctuations
    document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
    # Lowercase the numbers
    document_test = re.sub(r'[0-9]', '', document_test)
    # Remove the doubled space
    document_test = re.sub(r'\s{2,}', ' ', document_test)
    documents_clean.append(document_test)

## 3. DataFrame

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Instantiate a TfidfVectorizer object
vectorizer = TfidfVectorizer()
# It fits the data and transform it as a vector
X = vectorizer.fit_transform(documents_clean)
# Convert the X as transposed matrix
X = X.T.toarray()
# Create a DataFrame and set the vocabulary as the index
df = pd.DataFrame(X, index=vectorizer.get_feature_names())
df

,0,1,2,3,4,5,6,7,8,9
aamiin,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.079139,0.000000,0.000000
ada,0.040036,0.051849,0.019931,0.000000,0.050623,0.040559,0.086911,0.000000,0.000000,0.002667
adalah,0.000000,0.017583,0.000000,0.000000,0.000000,0.013754,0.014737,0.000000,0.000000,0.007234
adanya,0.000000,0.067815,0.000000,0.000000,0.000000,0.017683,0.000000,0.000000,0.000000,0.000000
adapun,0.034910,0.022605,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
yakin,0.000000,0.000000,0.000000,0.043222,0.044141,0.000000,0.000000,0.000000,0.000000,0.000000
yakni,0.000000,0.026591,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
yang,0.075914,0.108144,0.196525,0.018798,0.115186,0.138433,0.181278,0.029259,0.018689,0.004045
yg,0.000000,0.159547,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## 4. Get Article

In [4]:
#aku
def get_similar_articles(q, df):
  print("query:", q)
  print("Berikut artikel dengan nilai cosine similarity tertinggi: ")
  # Convert the query become a vector
  q = [q]
  q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
  sim = {}
  # Calculate the similarity
  for i in range(10):
    sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
  
  # Sort the values 
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
  # Print the articles and their similarity values
  for k, v in sim_sorted:
    if v != 0.0:
      print("Nilai Similaritas:", v)
      print(documents_clean[k])
      print()
# Add The Query
q1 = 'covid'
# Call the function
get_similar_articles(q1, df)

query: covid
Berikut artikel dengan nilai cosine similarity tertinggi: 
Nilai Similaritas: 0.1855739764418201
 kompas com unggahan terkait anggapan bahwa lonjakan kasus covid di indonesia terjadi setelah adanya program vaksinasi belakangan ini ramai di media sosial pertanyaan pertanyaan tersebut di antaranya diunggah netizen warganet di media sosial baik facebook maupun twitter salah satunya diunggah oleh akun facebook lazyboy baca juga mengenal apa itu ppkm darurat dan bedanya dengan ppkm mikro lur aku heran sebelum program vaksin kasus covid sudah mulai menurun tapi pada sadar nggak sehabis program vaksin malah banyak yang meninggal di desa ku banyak yg meninggal dan kebanyakan yg sudah pernah vaksin apa nggon ku tok tulisnya sejumlah warganet lain juga menanyakan hal tersebut di twitter dapatkan informasi inspirasi dan insight di email kamu daftarkan email nah gue setuju ini ada apa kenapa semakin banyak orang yg di vaksin malah kasus semakin meledak jangan kek gini lah pemerintah k